I am back with that good old dataset of MCQ distractor. 
Problem statement: We are given Question, Answer, and Distractors (1-3) for training and our job is to prepare distractor(at least 1 and at most 3) when given a question-ans pair.
Aproach: I in this kernel I will be using an attention based encoder-decoder model. I will use only the questions as input. And I will use the attention outputs from the source to generate word at each timestep which means I will not feed the output of last time step to geterate word. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mcq-distractors/Test.csv
/kaggle/input/mcq-distractors/Train.csv


In [2]:
# import packages
import os
import sys
import numpy as np
import pandas as pd
import string
import re
import tensorflow as tf
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from keras.layers import Input, Embedding, SimpleRNN, GRU, LSTM, Concatenate, Bidirectional, Dense, Flatten, Activation, Permute, RepeatVector, TimeDistributed, Dot, Multiply
from keras.models import Model
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
data = pd.read_csv('/kaggle/input/mcq-distractors/Train.csv').iloc[:,:3]
print(data.head())

                                            question  \
0                                Meals can be served   
1           It can be inferred from the passage that   
2     The author called Tommy 's parents in order to   
3           It can be inferred from the passage that   
4  How can we deal with snake wounds according to...   

                                         answer_text  \
0                             in rooms at 9:00 p. m.   
1  The local government can deal with the problem...   
2         help them realize their influence on Tommy   
3       the writer is not very willing to use idioms   
4                        Stay calm and do n't move .   

                                          distractor  
0                   'outside the room at 3:00 p. m.'  
1                   'If some tragedies occur again '  
2               'blame Tommy for his failing grades'  
3  'idioms are the most important part in a langu...  
4          'Cut the wound and suck the poison out .

In [4]:
# drop duplicates

data.drop_duplicates(inplace=True)

In [5]:
# remove quotes

data['question'] = data['question'].apply(lambda x: re.sub("'", '', x))
data['answer_text'] = data['answer_text'].apply(lambda x: re.sub("'", '', x))
data['distractor'] = data['distractor'].apply(lambda x: re.sub("'", '', x))
print(data.head())

                                            question  \
0                                Meals can be served   
1           It can be inferred from the passage that   
2      The author called Tommy s parents in order to   
3           It can be inferred from the passage that   
4  How can we deal with snake wounds according to...   

                                         answer_text  \
0                             in rooms at 9:00 p. m.   
1  The local government can deal with the problem...   
2         help them realize their influence on Tommy   
3       the writer is not very willing to use idioms   
4                         Stay calm and do nt move .   

                                         distractor  
0                    outside the room at 3:00 p. m.  
1                    If some tragedies occur again   
2                blame Tommy for his failing grades  
3  idioms are the most important part in a language  
4           Cut the wound and suck the poison out .  


In [6]:
# filter long texts

data['question_len'] = data['question'].apply(lambda x: len(x.split()))
data['answer_len'] = data['answer_text'].apply(lambda x: len(x.split()))
data['distractor_len'] = data['distractor'].apply(lambda x: len(x.split()))
print(len(data))
data = data[data['question_len']<=25]
data = data[data['answer_len']<=25]
data = data[data['distractor_len']<=25]
print(len(data))

# max question and answer length
max_question_len = 25
max_ans_len = 25 
max_dis_len = 26 # i will add START_ and _END # decoder input will have the START_ tokens and decoder output will have the _END

31499
31419


In [7]:
# convert to lower 

data['question'] = data['question'].apply(lambda x: x.lower())
data['answer_text'] = data['answer_text'].apply(lambda x: x.lower())
data['distractor'] = data['distractor'].apply(lambda x: x.lower())
print(data.head())

                                            question  \
0                                meals can be served   
1           it can be inferred from the passage that   
2      the author called tommy s parents in order to   
3           it can be inferred from the passage that   
4  how can we deal with snake wounds according to...   

                                         answer_text  \
0                             in rooms at 9:00 p. m.   
1  the local government can deal with the problem...   
2         help them realize their influence on tommy   
3       the writer is not very willing to use idioms   
4                         stay calm and do nt move .   

                                         distractor  question_len  answer_len  \
0                    outside the room at 3:00 p. m.             4           6   
1                    if some tragedies occur again              8          15   
2                blame tommy for his failing grades             9           7   
3 

In [8]:
# exclude special characters

special_char = string.punctuation
data['question'] = data['question'].apply(lambda x: ''.join(ch for ch in x if ch not in special_char))
data['answer_text'] = data['answer_text'].apply(lambda x: ''.join(ch for ch in x if ch not in special_char))
data['distractor'] = data['distractor'].apply(lambda x: ''.join(ch for ch in x if ch not in special_char))
print(data.head())

                                            question  \
0                                meals can be served   
1           it can be inferred from the passage that   
2      the author called tommy s parents in order to   
3           it can be inferred from the passage that   
4  how can we deal with snake wounds according to...   

                                         answer_text  \
0                                in rooms at 900 p m   
1  the local government can deal with the problem...   
2         help them realize their influence on tommy   
3       the writer is not very willing to use idioms   
4                          stay calm and do nt move    

                                         distractor  question_len  answer_len  \
0                       outside the room at 300 p m             4           6   
1                    if some tragedies occur again              8          15   
2                blame tommy for his failing grades             9           7   
3 

In [9]:
# add START_ and _END tokens

data['distractor'] = data['distractor'].apply(lambda x: 'START_ '+x+' _END')
print(data.head())

                                            question  \
0                                meals can be served   
1           it can be inferred from the passage that   
2      the author called tommy s parents in order to   
3           it can be inferred from the passage that   
4  how can we deal with snake wounds according to...   

                                         answer_text  \
0                                in rooms at 900 p m   
1  the local government can deal with the problem...   
2         help them realize their influence on tommy   
3       the writer is not very willing to use idioms   
4                          stay calm and do nt move    

                                          distractor  question_len  \
0            START_ outside the room at 300 p m _END             4   
1         START_ if some tragedies occur again  _END             8   
2     START_ blame tommy for his failing grades _END             9   
3  START_ idioms are the most important part i

In [10]:
# creating question and answer vocabulary

all_question_words = []
for i in data['question']:
    all_question_words += i.split()

all_question_words = list(set(all_question_words))

all_ans_words = []
for i in data['answer_text']:
    all_ans_words += i.split()
    
all_ans_words = list(set(all_ans_words))

all_dis_words = []
for i in data['distractor']:
    all_dis_words += i.split()
    
all_dis_words = list(set(all_dis_words))

In [11]:
source_words = sorted(list(set(all_question_words +all_ans_words)))
target_words = sorted(all_dis_words)

num_source_tokens = len(source_words)
num_target_tokens = len(target_words)

In [12]:
# tokenizing words

source_tokens = dict([(word, i+1) for i, word in enumerate(source_words)])
target_tokens = dict([(word, i+1) for i, word in enumerate(target_words)])

In [13]:
# reverse tokenizing

reverse_source_tokens = dict([(i,word) for word, i in source_tokens.items()])
reverse_target_tokens = dict([(i,word) for word, i in target_tokens.items()])

In [14]:
# train and test data 

source1, source2, target = data['question'], data['answer_text'], data['distractor']
source1_train, source1_test, source2_train, source2_test, target_train, target_test = train_test_split(source1, source2, target, test_size=0.2, random_state=21)

In [15]:
# function to generate batches as the entire data is to large to store at once. 
def generate_batch(X1, y, batch_size=64):
    while True:
        s0 = np.zeros((batch_size, latent_dim))
        c0 = np.zeros((batch_size, latent_dim))
        for b in range(0, len(X1), batch_size):
            encoder_source1_train = np.zeros((batch_size, max_question_len), dtype='float32')
            encoder_source2_train = np.zeros((batch_size, max_ans_len), dtype='float32')
            decoder_source_train = np.zeros((batch_size, max_dis_len), dtype='float32')
            decoder_target_train = np.zeros((batch_size, max_dis_len, num_target_tokens))
            for i, (source1_text, target_text) in enumerate(zip(X1[b:b+batch_size], y[b:b+batch_size])):
                for t, word in enumerate(source1_text.split()):
                    encoder_source1_train[i, t] = source_tokens[word]
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_source_train[i, t] = target_tokens[word]
                    if t>0:
                        decoder_target_train[i, t-1, target_tokens[word]-1] = 1
            yield ([encoder_source1_train,s0, c0], list(decoder_target_train.swapaxes(0,1)))
 

In [16]:
repeator = RepeatVector(25)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")    
densor2 = Dense(1, activation = "relu")

def one_step_attention(a, s_prev):
    activator = Activation('softmax') 
    dotor = Dot(axes = 1)
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    e = densor1(concat)
    energies = densor2(e) 
    alphas = activator(energies)
    context = dotor([alphas,a])
    
    return context

In [17]:
latent_dim = 300
s0 = Input(shape=(latent_dim,), name='s0')
c0 = Input(shape=(latent_dim,), name='c0')
s = s0
c = c0
units = 50
# encoder
encoder_input1 = Input(shape=(25,))
#print(encoder_input1.get_shape()) 
encoder_embedding = Embedding(num_source_tokens, latent_dim, mask_zero=False)
source1_encoded_embedding = encoder_embedding(encoder_input1)
encoder_LSTM = LSTM(latent_dim, return_sequences=True)
source1_encoder_outputs = encoder_LSTM(source1_encoded_embedding)
#print('Encoder done')
outputs = []
post_activation_LSTM_cell = LSTM(latent_dim, return_state = True)
output_layer = Dense(num_target_tokens, activation='softmax')
for t in range(26):
    context = one_step_attention(source1_encoder_outputs,s)
    #print("Context"+str(t))
    s, _, c = post_activation_LSTM_cell(context, initial_state = [s,c])
    out = output_layer(s)
    outputs.append(out)

print(len(outputs))  
model = Model(inputs = [encoder_input1,s0,c0], outputs = outputs)
model.summary()



26
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 300)      5685300     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 300)          0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 25, 300)      721200      embedding_1[0][0]                
_________________________________________________________________________________________

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [19]:
model.fit_generator(generator=generate_batch(source1_train, target_train, batch_size=64),steps_per_epoch = len(source1_train)//64, validation_data=generate_batch(source1_test, target_test, batch_size=64), validation_steps=len(source1_test)//64, epochs = 25)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/25
392/392 [==============================] - 134s 342ms/step - loss: 56.3643 - dense_3_loss: 0.0000e+00 - val_loss: 57.0623 - val_dense_3_loss: 0.0000e+00
Epoch 2/25
392/392 [==============================] - 112s 287ms/step - loss: 55.6258 - dense_3_loss: 0.0000e+00 - val_loss: 50.8201 - val_dense_3_loss: 0.0000e+00
Epoch 3/25
392/392 [==============================] - 112s 285ms/step - loss: 54.8910 - dense_3_loss: 0.0000e+00 - val_loss: 50.1123 - val_dense_3_loss: 0.0000e+00
Epoch 4/25
392/392 [==============================] - 112s 285ms/step - loss: 54.3717 - dense_3_loss: 0.0000e+00 - val_loss: 57.3761 - val_dense_3_loss: 0.0000e+00
Epoch 5/25
392/392 [==============================] - 113s 288ms/step - loss: 53.9639 - dense_3_loss: 0.0000e+00 - val_loss: 53.1379 - val_dense_3_loss: 0.0000e+00
Epoch 6/25
392/392 [==============================] - 111s 282ms/step - loss: 53.5623 - dense_3_loss: 0.0000e+00 - val_loss: 55.9959 - val_dense_3_loss: 0.0000e+00
Epoch 7/25
392/3